In [1]:
import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig

#get existing workspace
ws = Workspace.from_config()
ws

In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
#compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'cpu-cluster')
compute_name = 'amlcluster'
#compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
compute_min_nodes = 0
#compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MAX_NODES', 4)
compute_max_nodes = 4

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
#vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'STANDARD_D2_V2')
vm_size = 'STANDARD_D2_V2'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes, 
                                                                max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. amlcluster


In [3]:
from azureml.core.dataset import Dataset

train_data = Dataset.get_by_name(ws,"iris")

pdtrain_data = train_data.to_pandas_dataframe()

pdtrain_data.shape

(150, 5)

In [5]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig

import logging

experiment_name = 'iris-automl'

target_column_name = 'species'

experiment = Experiment(ws, experiment_name)

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_debug_logs.log',
                             primary_metric='accuracy',
                             experiment_timeout_minutes=10,
                             preprocess=True,
                             max_concurrent_iterations=4,
                             max_cores_per_iteration=-1,
                             enable_dnn=True,
                             enable_early_stopping=True,
                             validation_size=0.3,
                             verbosity = logging.INFO,                             
                             compute_target=compute_target,
                             training_data=train_data,
                             label_column_name=target_column_name)

remote_run = experiment.submit(automl_config, show_output=True)


Running on remote compute: amlcluster
Parent Run ID: AutoML_c29683ec-b226-4f7d-84f7-0835d791949c

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         3   MinMaxScaler RandomForest                      0:10:05       0.9556    0.9556
         2   MinMaxScaler SGD                               0:05:58       0.6667    0.9556
         1   StandardScalerWrapper S

In [6]:
from azureml.widgets import RunDetails

# monitor the run
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [7]:
remote_run.wait_for_completion()

# Retrieve the best model to use it further.
_, fitted_model = remote_run.get_output()

In [8]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
iris-automl,AutoML_c29683ec-b226-4f7d-84f7-0835d791949c,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
best_run, fitted_model = remote_run.get_output()
model_name = best_run.properties['model_name']

In [10]:
model_name

'AutoMLc29683ecb8'

In [11]:
#Register Model
description = 'AutoML Iris V1'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id)

AutoMLc29683ecb8


In [12]:
script_file_name = 'score_fcast.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

conda_file_name = 'conda_file.yml'
best_run.download_file('outputs/conda_env_v_1_0_0.yml', conda_file_name)

#automlenv = Environment.from_conda_specification(name = "automlenv", file_path = conda_file_name)


In [13]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

conda = CondaDependencies.create(conda_packages=['numpy>=1.16.0,<=1.16.2','pandas','scikit-learn','py-xgboost<=0.80','fbprophet==0.5','psutil>=5.2.2,<6.0.0'],pip_packages=['azureml-defaults==1.0.83','azureml-train-automl-runtime==1.0.83.1','inference-schema','azureml-explain-model==1.0.83'])

myenv=Environment(name="irisenv")
myenv.python.conda_dependencies = conda



In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (can also provide parameters to customize)
#prov_config = AksCompute.provisioning_configuration()

aks_name = 'MTCAKSCluster' 
# Create the cluster
#aks_target = ComputeTarget(workspace = workspace,name = aks_name)

aks_target = AksCompute(workspace=ws,name=aks_name)

aks_config = AksWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,auth_enabled=True)


# inference_config = InferenceConfig(environment= automlenv,
#                                     entry_script = script_file_name)


inference_config = InferenceConfig(environment=myenv,
                                     entry_script = script_file_name)



#inference_config = InferenceConfig(entry_script = script_file_name)


api_service_name = 'automl-iris-api'

api_service = Model.deploy(workspace=ws,
                           name=api_service_name, 
                           models=[model],
                           inference_config=inference_config, 
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                          overwrite=True)

api_service.wait_for_deployment(show_output=True)
print(api_service.state)

In [14]:
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

input_sample = pd.DataFrame(data=[{'sepal_length': 5.9, 'sepal_width': 3.0, 'petal_length': 5.1, 'petal_width': 1.8}])


new_model_name = 'bestmodel.pkl'
best_run.download_file('outputs/model.pkl', new_model_name)

model_path = Model.get_model_path(new_model_name)

model = joblib.load(model_path)

data = input_sample

result = model.predict(data)

json.dumps({"result": result.tolist()})
    
#display(result)


'{"result": ["Iris-virginica"]}'

In [ ]:

import json

test_sample = json.dumps({'data': [
    [5.9,3.0,5.1,1.8]
]})

api_service.run(input_data=test_sample)